---
title: "自己动手从零开始写语言模型 (二): 文本数据的处理方法"
date: 2025-07-20T09:05:03-05:00
author: "郝鸿涛"
slug: llm
draft: true
toc: false
tags: llm
---

计算机无法直接处理文本，我们需要把文本信息转换为计算机可以理解的数字。怎么做呢？你可以这样想：我们是否可以给每个字在多维空间中找到一个坐标。我们的目标是，相近的词，坐标要接近。这个方法叫做 Word Emebedding。

如果把这个多维空间中的许多坐标投影到二维，我们希望看到的结果是：

![Word Embedding](img/word_embedding.png)

当然，一个词可以有位置，那我们也可以给每个句子、段落、文章找到位置。这些在 retrieval-augmented generation (RAG) 中比较常用，比如每个电影的介绍，我们确定一个位置，然后用户搜索后，确定搜索文本的位置，然后找到和这个搜索文本位置最接近的电影介绍，最后喂给 LLM，输出结果。具体实现可以看我的[这篇博客](/cn/2025/03/16/bert/)。

下面，我们用一篇文章来展示处理文本的主要步骤。

In [22]:
import urllib.request

# 下载文本信息
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "data/the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

# 读取
with open("data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [23]:
print("Total number of character:", len(raw_text))

Total number of character: 20479


In [24]:
# 打印前 199 个字符
print(raw_text[:199])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married 


## 文本到 ID 相互转换

之前提到过，计算机只能读取数字。我们需要给每一个 token 安排一个 ID。比如，我们说 I 对应 0, always 对应 1 等等。你可以把这个想象成一个字典或者电台密码本。但有个问题是，小的文本我们可以给每个词一个 id，但是在大语言模型训练中，肯定会出现新的词，以及很奇怪的词，那怎么办？比如，一篇文章里突然出现了 SomeStrangeWordYouHaveNeverSeenBefore 这个词，怎么办？

有一个办法是所有字典里没有的词，统一用 `<|unk|>` 来代替，这里提一句，还有一个非常重要的 id 是 `<|endoftext|>`，用来指示一段文本的结束。这可以在训练时告诉 LLM，下面内容来自不同的来源。

用 `<|unk|>` 的弊端至少有两个：1. 错过一些有意义的陌生词，比如我上面举的那个例子。也就是 LLM 训练时，我们没办法正确给它位置，因为我们看不到它的内容。2. 从 ID 转换到文本这个过程会失败。比如，我们知道 1 对应 always，但是我们肯定无法根据 ID 转换回 SomeStrangeWordYouHaveNeverSeenBefore。

一个更有效的方法是 byte pair encoding (BPE)，也是 GPT 训练时用的方法。我们会用到 OpenAI 开发的包 [tiktoken](https://github.com/openai/tiktoken)。

在 Terminal 输入：

```sh
pip install tiktoken
```

然后正常使用：

In [25]:
import tiktoken
# 初始化
tokenizer = tiktoken.get_encoding('gpt2')

# 测试从文本到 token ID
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces "
    "of someunknownPlace and SomeStrangeWordYouHaveNeverSeenBefore."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 290, 2773, 38114, 26449, 1639, 11980, 12295, 4653, 268, 8421, 13]


In [26]:
# 检测从 ID 回到文本
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace and SomeStrangeWordYouHaveNeverSeenBefore.


我们看到转换很顺利。BEP 的机制如下：

![byte pair encoding](img/byte_pair_encoding.png)

## 准备训练数据

还记得我们之前提到过的 LLM 训练方法吗？根据之前的文本，预测下一个词：

![LLM Prediction](img/llm_predict.png)

那训练数据是不是要这样子：

In [27]:
enc_text = tokenizer.encode(raw_text)
enc_sample = enc_text[50:] # 此步非必须
context_size = 4 

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "----->", desired)

[290] -----> 4920
[290, 4920] -----> 2241
[290, 4920, 2241] -----> 287
[290, 4920, 2241, 287] -----> 257


In [28]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "----->", tokenizer.decode([desired]))

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


我们的最终目标是得到 inputs tensor 和 target tensor:

![Data Loader](img/data_loader.png)

这里有两个问题，或者说变量。第一个是每个 vector 有多大？我们用 `max_length` 表示。这里显示是 4。但这只是为了显示，在真正的 LLM 训练中，`max_length >= 256`。

另一个变量是 `stride`: 一句话结束之后，我们下一句的开始在哪里？

![Stride illustration](img/stride.png)